In [52]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier


In [53]:
test_df=pd.read_csv("test.csv")
train_df=pd.read_csv("train.csv")

In [54]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [55]:
train_df.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [56]:
train_df.head(10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


In [57]:
total=train_df.isnull().sum().sort_values(ascending=False)
percent1=train_df.isnull().sum()/train_df.isnull().count()*100
percent2=round(percent1,1).sort_values(ascending=False)
missing_data=pd.concat([total,percent2],axis=1,keys=['Total','%'])
missing_data.head(10)

,Total,%
Cabin,687,77.1
Age,177,19.9
Embarked,2,0.2
Fare,0,0.0
Ticket,0,0.0
Parch,0,0.0
SibSp,0,0.0
Sex,0,0.0
Name,0,0.0
Pclass,0,0.0


In [58]:
train_df.columns.values

array(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'], dtype=object)

In [59]:
data=[train_df,test_df]
for dataset in data:
    dataset['relatives']=dataset['SibSp']+dataset['Parch']
    dataset.loc[dataset['relatives']>0 ,'not_alone']=0
    dataset.loc[dataset['relatives']==0,'not_alone']=1
    # dataset['not_alone']=dataset['not_alone'].astype(int)
train_df['not_alone'].value_counts()

1.0    537
0.0    354
Name: not_alone, dtype: int64

In [60]:
train_df=train_df.drop(['PassengerId'],axis=1)

In [61]:
import re
deck={"A":1, "B":2 ,"C":3, "D":4, "E":5, "F":6, "G":7, "U":8}
data=[train_df,test_df]

for dataset in data:
    dataset['Cabin']=dataset['Cabin'].fillna("U0")
    dataset['Deck']=dataset['Cabin'].map(lambda x:re.compile("([a-zA-Z]+)").search(x).group())
    dataset['Deck']=dataset['Deck'].map(deck)
    dataset['Deck']=dataset['Deck'].fillna(0)
    dataset['Deck']=dataset['Deck'].astype(int)
    
train_df=train_df.drop(['Cabin'], axis=1)
test_df=test_df.drop(['Cabin'], axis=1)

In [62]:
data =[train_df,test_df]

for dataset in data:
    mean=train_df["Age"].mean()
    std = test_df["Age"].std()
    is_null=dataset["Age"].isnull().sum()
    rand_age=np.random.randint(mean-std, mean+std, size=is_null)
    age_slice =dataset["Age"].copy()
    age_slice[np.isnan(age_slice)]=rand_age
    dataset["Age"]= age_slice
    dataset["Age"]=train_df["Age"].astype(int)
    
train_df["Age"].isnull().sum()

0

In [63]:
train_df["Embarked"].describe()

count     889
unique      3
top         S
freq      644
Name: Embarked, dtype: object

In [64]:
common_values ='S'
data = [train_df, test_df]
for dataset in data:
    dataset["Embarked"] = dataset["Embarked"].fillna(common_values)

In [65]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 13 columns):
Survived     891 non-null int64
Pclass       891 non-null int64
Name         891 non-null object
Sex          891 non-null object
Age          891 non-null int32
SibSp        891 non-null int64
Parch        891 non-null int64
Ticket       891 non-null object
Fare         891 non-null float64
Embarked     891 non-null object
relatives    891 non-null int64
not_alone    891 non-null float64
Deck         891 non-null int32
dtypes: float64(2), int32(2), int64(5), object(4)
memory usage: 83.7+ KB


In [66]:
data = [train_df, test_df]

for dataset in data:
    dataset['Fare']=dataset['Fare'].fillna(0)
    dataset['Fare']=dataset['Fare'].astype(int)

In [67]:
data =[train_df, test_df]
titles = {"Mr": 1, "Miss": 2, "Mrs": 3, "Master": 4, "Rare": 5}

for dataset in data:
    dataset['Title']=dataset.Name.str.extract('([A-Za-z]+)\.',expand=False)
    #replacing titles
    dataset['Title']=dataset['Title'].replace(['Lady','Countess', 'Capt', 'Col', 'Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Done' ],'Rare')
    dataset['Title']=dataset['Title'].replace('Ms','Miss')
    dataset['Title']=dataset['Title'].replace('Mme', 'Mrs')
    dataset['Title']=dataset['Title'].replace('Mlle', 'Miss')
    #converting titles into numbers
    dataset['Title']=dataset['Title'].map(titles)
    dataset['Title']=dataset['Title'].fillna(0)
    
train_df = train_df.drop(['Name'], axis=1)
test_df = test_df.drop(['Name'], axis=1)

In [68]:
genders = {"male": 0, "female": 1}
data = [train_df, test_df]

for dataset in data:
    dataset['Sex'] = dataset['Sex'].map(genders)

In [69]:
train_df['Ticket'].describe()

count      891
unique     681
top       1601
freq         7
Name: Ticket, dtype: object

In [70]:
train_df = train_df.drop(['Ticket'], axis=1)
test_df = test_df.drop(['Ticket'], axis=1)

In [71]:
ports = {"S": 0, "C": 1, "Q": 2}
data=[train_df, test_df]

for dataset in data:
    dataset['Embarked']=dataset['Embarked'].map(ports)

In [72]:
data = [train_df, test_df]
for dataset in data:
    dataset['Age'] = dataset['Age'].astype(int)
    dataset.loc[(dataset['Age']<=11), 'Age']=0
    dataset.loc[(dataset['Age']> 11) & (dataset['Age'] <= 18), 'Age'] = 1
    dataset.loc[(dataset['Age']> 18) & (dataset['Age'] <= 22), 'Age'] = 2
    dataset.loc[(dataset['Age']> 22) & (dataset['Age'] <= 27), 'Age'] = 3
    dataset.loc[(dataset['Age']> 27) & (dataset['Age'] <= 33), 'Age'] = 4
    dataset.loc[(dataset['Age']> 33) & (dataset['Age'] <= 40), 'Age'] = 5
    dataset.loc[(dataset['Age']> 40) , 'Age'] = 6
train_df['Age'].value_counts()

4    172
6    158
5    147
3    132
2    120
1     94
0     68
Name: Age, dtype: int64

In [73]:
pd.qcut(train_df['Fare'], q=7
       )

0      (-0.001, 7.0]
1      (56.0, 512.0]
2      (-0.001, 7.0]
3       (27.0, 56.0]
4         (7.0, 8.0]
           ...      
886     (12.0, 19.0]
887     (27.0, 56.0]
888     (19.0, 27.0]
889     (27.0, 56.0]
890    (-0.001, 7.0]
Name: Fare, Length: 891, dtype: category
Categories (7, interval[float64]): [(-0.001, 7.0] < (7.0, 8.0] < (8.0, 12.0] < (12.0, 19.0] < (19.0, 27.0] < (27.0, 56.0] < (56.0, 512.0]]

In [74]:
train_df['Fare'].describe()

count    891.000000
mean      31.785634
std       49.703730
min        0.000000
25%        7.000000
50%       14.000000
75%       31.000000
max      512.000000
Name: Fare, dtype: float64

In [75]:
data = [train_df, test_df]

for dataset in data:
    dataset.loc[dataset['Fare'] <= 7, 'Fare'] = 0
    dataset.loc[(dataset['Fare'] > 7) &(dataset['Fare']<=8), 'Fare']=1
    dataset.loc[(dataset['Fare'] > 8) &(dataset['Fare']<=14), 'Fare']=2
    dataset.loc[(dataset['Fare'] > 14) &(dataset['Fare']<=26), 'Fare']=3
    dataset.loc[(dataset['Fare'] > 26) &(dataset['Fare']<=52), 'Fare']=4
    dataset.loc[(dataset['Fare'] > 52), 'Fare']=5
    dataset['Fare']=dataset['Fare'].astype(int)

In [76]:
data =[train_df, test_df]
for dataset in data:
    dataset['Age_Class'] = dataset['Age']*dataset['Pclass']

In [77]:
for dataset in data:
    dataset['Fare_Per_Person'] = dataset['Fare']/(dataset['relatives']+1)
    dataset['Fare_Per_Person'] = dataset['Fare_Per_Person'].astype(int)

In [78]:
train_df.head(10)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,relatives,not_alone,Deck,Title,Age_Class,Fare_Per_Person
0,0,3,0,2,1,0,0,0,1,0.0,8,1,6,0
1,1,1,1,5,1,0,5,1,1,0.0,3,3,5,2
2,1,3,1,3,0,0,0,0,0,1.0,8,2,9,0
3,1,1,1,5,1,0,5,0,1,0.0,3,3,5,2
4,0,3,0,5,0,0,1,0,0,1.0,8,1,15,1
5,0,3,0,6,0,0,1,2,0,1.0,8,1,18,1
6,0,1,0,6,0,0,4,0,0,1.0,5,1,6,4
7,0,3,0,0,3,1,3,0,4,0.0,8,4,0,0
8,1,3,1,3,0,2,2,0,2,0.0,8,3,9,0
9,1,2,1,1,1,0,4,1,1,0.0,8,3,2,2


In [79]:
X_train = train_df.drop("Survived", axis=1)
Y_train = train_df['Survived']
X_test = test_df.drop("PassengerId", axis=1).copy()

In [80]:
random_forest = RandomForestClassifier( n_estimators=100, oob_score = True)
random_forest.fit(X_train, Y_train)

Y_prediction = random_forest.predict(X_test)

random_forest.score(X_train, Y_train)
acc=round(random_forest.score(X_train, Y_train) * 100, 2)
print('accuracy=',acc,'oob_score=',random_forest.oob_score_)

accuracy= 92.82 oob_score= 0.813692480359147


In [81]:
from sklearn.model_selection import cross_val_score
rf = RandomForestClassifier( n_estimators=100)
scores = cross_val_score(rf, X_train, Y_train, cv=10, scoring="accuracy")

print("Scores:", scores)
print("Mean:", scores.mean())
print("Standard Deviation:", scores.std())

Scores: [0.77777778 0.82222222 0.75280899 0.82022472 0.87640449 0.84269663
 0.84269663 0.75280899 0.83146067 0.82954545]
Mean: 0.814864657814096
Standard Deviation: 0.038713634683711626


In [ ]:
importances = pd.DataFrame({'feature':X_train.columns, 'importance':np.round(random_forest.feature_importances_, 3)})
importances = importances.sort_values('importance', ascending = False).set_index('feature')
importances.head(15)

In [83]:
Y_prediction.shape


(418,)

In [84]:
train_df = train_df.drop('not_alone', axis=1)
test_df = test_df.drop('not_alone', axis=1)

train_df = train_df.drop('Parch', axis=1)
test_df = test_df.drop('Parch', axis=1)


In [85]:
random_forest = RandomForestClassifier( n_estimators=100, oob_score = True)
random_forest.fit(X_train, Y_train)

Y_prediction = random_forest.predict(X_test)

random_forest.score(X_train, Y_train)
acc=round(random_forest.score(X_train, Y_train) * 100, 2)
print('accuracy=',acc,'oob_score=',random_forest.oob_score_)

accuracy= 92.82 oob_score= 0.8103254769921436


In [88]:
param_grid = { "criterion": ["gini", "entropy"], "min_samples_leaf": [1, 5, 10, 25, 50, 70 ], "min_samples_split": [2, 4, 10, 12, 16, 18, 25, 35], "n_estimators": [100, 400, 700, 1000, 1500]}

from sklearn.model_selection import GridSearchCV, cross_val_score

rf=RandomForestClassifier(n_estimators=100, max_features='auto', oob_score=True, random_state=1, n_jobs=-1)

clf =GridSearchCV(estimator = rf, param_grid= param_grid, n_jobs=-1)

clf.fit(X_train, Y_train)

clf.bestparams

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


KeyboardInterrupt: 

In [102]:
random_forest = RandomForestClassifier(criterion="gini", min_samples_leaf = 1, min_samples_split = 10, n_estimators = 100, max_features='auto', oob_score=True, random_state=1, n_jobs=-1)

random_forest.fit(X_train, Y_train)
Y_prediction = random_forest.predict(X_test)

random_forest.score(X_train, Y_train)
print("oob score:", round(random_forest.oob_score_, 4)*100, "%" )

oob score: 82.6 %


In [103]:
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
predictions = cross_val_predict(random_forest, X_train, Y_train, cv=3)
confusion_matrix(Y_train, predictions)

array([[488,  61],
       [ 97, 245]], dtype=int64)

In [104]:
from sklearn.metrics import precision_score, recall_score

print("precision:", precision_score(Y_train, predictions))
print("Recall:", recall_score(Y_train, predictions))

precision: 0.8006535947712419
Recall: 0.716374269005848


In [105]:
from sklearn.metrics import precision_recall_curve

# getting the probabilities of our predictions
y_scores = random_forest.predict_proba(X_train)
y_scores = y_scores[:,1]

precision, recall, threshold = precision_recall_curve(Y_train, y_scores)
from sklearn.metrics import roc_auc_score
r_a_score = roc_auc_score(Y_train, y_scores)
print("ROC-AUC-Score:", r_a_score)

ROC-AUC-Score: 0.9464017511903621


In [96]:
submission = pd.DataFrame({'PassengerId':test_df['PassengerId'],'Survived':Y_prediction})

submission.head(10)

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,0
5,897,0
6,898,1
7,899,0
8,900,1
9,901,0


In [97]:
filename = 'Titanic Predictions 2.csv'

submission.to_csv(filename, index=False)

print('Saved file:',filename)

Saved file: Titanic Predictions 2.csv
